In [1]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# @Title: ArcGIS Online Administration Notebook
#
# @Purpose: this notebook is a sample of how the ArcGIS Python
# API can be used to adminster a portal. Specifically, this
# notebook generates a report of: 
# 1) items in the portal, including those with the user-defined words in the tag or the title
# 2) user status based on last login and activities
# 3) Hosted feature services used in the org
# The reports update a hosted table connected to a dashboard for interactive exploration. 
#
# @Last Updated: January 2023
# @Reference the Blog!: 'https://www.esri.com/arcgis-blog/products/arcgis-online/administration/managing-arcgis-online-content-with-arcgis-dashboards-and-arcgis-notebooks/'
#
# @Versions: ArcGIS Python API v1.8.4
#
# @License: See end of notebook
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

"''\n# @Title: ArcGIS Online Administration Notebook\n#\n# @Purpose: this notebook is a sample of how the ArcGIS Python\n# API can be used to adminster a portal. Specifically, this\n# notebook generates a report of: \n# 1) items in the portal, including those with the word 'test' in the tag or the title\n# 2) user status based on last login and activities\n# 3) Hosted feature services used in the org\n# The reports update a hosted table connected to a dashboard for interactive exploration. \n#\n# @Creator: ckwon@esri.com\n# @Credits: Building upon work done by Geospatial Center SEs\n# @Last Updated: April 2021\n#\n# @Versions: ArcGIS Python API v1.8.4\n#\n# @License: See end of notebook\n"

# Import Libraries & Connect to the portal

In [2]:
#Import relevant libraries
import io
import csv
import time
import getpass
import requests
import pandas as pd
import datetime
from arcgis.features import FeatureLayer
from arcgis.gis import GIS, Item, User
from arcgis.gis.admin import License, LicenseManager

# User Inputs

In [3]:
#Provide item ID of Hosted Feature Tables
#Reference the instructions outlined in the blog
table_item = 'INPUT YOUR TABLE'
table_user = 'INPUT YOUR TABLE'
table_fs = 'INPUT YOUR TABLE'

#Number of days to keep historical data. Select '0' to not retain any data. 
#Recommendation is to keep this 30 days or less.
days = 15

#Keywords to search in title & tag. Put them in a list
words = ['test','sample','draft']

In [5]:
#Connect to the GIS environment
gis = GIS("home")

# Item Administration

In [6]:
#From the REST URL, request all items in the organization. 
#Create a dictionary of the results, and print out the output. 
url = f'{gis.url}/sharing/rest/content/portals/{gis.properties.id}'

params = {
    'f': 'csv',
    'token': gis._portal.con.token
}

#Get a string response from the request and construct a DataFrame
csv_out = requests.get(url, params=params).text
df = pd.read_csv(io.StringIO(str(csv_out)))

#Replace NaN values in the df as presence of NaN results in error when adding features to the feature table
df_filled = df.fillna('None')
df_filled

,id,title,owner,fullname,size,created,modified,url
0,c13d470a1c024318ad221d75412649b4,SIDW_Forum_Sponsors,ssawaya_aid,Salim Sawaya,11105,2013-06-02 16:57:52.0,2021-03-22 19:41:53.0,None
1,d3bf9513eb6148e99df2436563fe3167,SIDW_Forum_Sponsors,ssawaya_aid,Salim Sawaya,57344,2013-06-02 16:57:55.0,2013-06-06 06:07:16.0,http://services.arcgis.com/LG9Yn2oFqZi5PnO5/ar...
2,8bfad945962f4d40b0ae7ce7f74293e3,SID-W 2013 Sponsors HQs,ssawaya_aid,Salim Sawaya,2781,2013-06-02 17:09:41.0,2013-06-06 17:02:16.0,None
3,b6f3ab78228f45578427ee1f055b0995,SID-W 2013 Sponsors,ssawaya_aid,Salim Sawaya,488,2013-06-02 17:11:19.0,2013-06-02 21:19:29.0,http://esriaiddev.maps.arcgis.com/apps/SocialM...
4,cb2cbb56c69746b0b5b29753c6bf404b,AgMarketFinder,ssawaya_aid,Salim Sawaya,0,2013-06-03 01:41:47.0,2013-06-03 05:45:51.0,http://marketfinder.info/
...,...,...,...,...,...,...,...,...
5479,67ed2ed54df4463f858ff5b6d0bdf93d,MSI AMELIA,elovell_aid,Eric Lovell,14,2021-04-08 15:47:35.0,2021-04-08 15:47:35.0,None
5480,095b2820e8384ac2afed3049650a86c9,Affected Population 2020 2021,ralouta.aiddev,Rami Alouta,404441,2021-04-08 15:59:11.0,2021-04-08 15:59:11.0,None
5481,b4505f385c0a4a5ab6c833446c03bb3e,Affected Population 2020 2021,ralouta.aiddev,Rami Alouta,3366912,2021-04-08 15:59:13.0,2021-04-08 15:59:43.0,https://services.arcgis.com/LG9Yn2oFqZi5PnO5/a...
5482,871f2e4fcd7f4807ae76583b0d4adf8f,GeoJournalism_WFL1_1_View,ckwon_aid,Calvin Kwon,0,2021-04-08 16:31:17.0,2021-04-08 16:39:26.0,https://services.arcgis.com/LG9Yn2oFqZi5PnO5/a...


In [ ]:
#Iterate through each record, add new item attributes, and determine if 'test' exists in the title & the tag.

f_item = []
attributeMap = ['id','title','owner','fullname','created','modified','url','size']

for index, row in df_filled.iterrows():
    #Create an empty dictionary for each item
    f = {"attributes":{}}
    
    #Write the row of each record to the dictionary
    for field in attributeMap:
        f['attributes'][field] = row[field]
    
        #Update for November 2022 Update of ArcGIS Online - Thank you Esri Canada!
        if field == 'created' or field == 'modified':
            # Take the new date format, and convert it to a datetime type
            newDate = datetime.datetime.strptime(row[field], "%a %b %d %H:%M:%S %Z %Y")
            # Once converted, pass it to the f_items JSON dictionary
            f['attributes'][field] = newDate
        else:
            f['attributes'][field] = row[field]

    #Then convert size in bytes to megabytes, format to six decimal places & update
    size_mb = int(row['size'])/1000000
    f['attributes']['size_mb'] = "{:.6f}".format(size_mb)
    
    #Get the item and include additional fields
    #Documentation on available fields: https://developers.arcgis.com/rest/users-groups-and-items/common-parameters.htm#ESRI_SECTION1_1FFBA7FE775B4BDA8D97524A6B9F7C98
    item = gis.content.get(itemid=row['id'])
    f['attributes']['access'] = item['access']
    f['attributes']['type'] = item['type']
    f['attributes']['snippet'] = item['snippet']
    f['attributes']['categories'] = str(item['categories']).strip('[]') #Convert to string & remove the []. A list cannot be inserted into a string field in the hosted table
    f['attributes']['numComments'] = item['numComments']
    f['attributes']['numRatings'] = item['numRatings']
    f['attributes']['numViews'] = item['numViews']
    f['attributes']['scoreCompleteness'] = item['scoreCompleteness']
    f['attributes']['tags'] = str(item['tags']).strip('[]') #Convert to string & remove the []. A list cannot be inserted into a string field in the hosted table
    
    #For each item, check if the title or tag contains the word 'test'
    #First check if the item has a title. If it does not, indicate the item. 
    #Then check if the title has the word 'test,' followed by the tag
    #Assign a variable for title, tag, and id to simplify logic below
    item_title = f['attributes']['title']
    item_tag = f['attributes']['tags']
    item_id = f['attributes']['id']
    
    if isinstance(item_title, float): 
        print(f'Item ID:{item_id} does not have a valid title')
    else: 
        if any(x in item_title for x in words):
            f['attributes']['tag_status'] = 'yes'
        elif item_tag != '':
            if any(x in item_tag for x in words):
                f['attributes']['tag_status'] = 'yes'
            else:
                f['attributes']['tag_status'] = 'no'            
        else:
            f['attributes']['tag_status'] = 'no'            
    
    #Append the record to the list 
    f_item.append(f)

#Display the first 5 items as output
f_item[:1]

In [ ]:
#Count the number of items where tag_status = 'yes' for reporting purposes  
count_yes = 0
count_no = 0

for record in f_item:
    tag_status = record.get('attributes',{}).get('tag_status')
    if tag_status == 'yes':
        count_yes += 1
    else:
        count_no += 1

count_total = count_yes + count_no
        
print(f'{count_total} items were checked...')
print(f'{count_yes} items have the word "test" in the title or the tag.')

In [ ]:
#If this is the first time running the notebook, create the hosted table with the following CSV. 
#https://esriis-my.sharepoint.com/:x:/g/personal/cal10660_esri_com/EYi8huq-F25AipgAONwVK5IB5NOOIFop9WaHXteq9TGJSw?e=ZCdFNa

#Make sure to set the field types correctly.
#String: id, title, owner, fullname, url, access, type, snippet, categories, tags, tag_status
#Integer: numComments, numRatings, numViews, scoreCompleteness
#Double: size, size_mb
#Date: created, modified

dest_fl = gis.content.get(table_item).tables[0]
dest_fl

In [ ]:
#Update the Hosted Table with the item details
dest_fl.delete_features(where="1=1")
result = dest_fl.edit_features(adds=f_item)
print('success...')

# User Administration

In [11]:
#Search for users in the organization 
search = 1000 #Make sure this value exceeds the no. of users in the org
users_all = gis.users.search(max_users=search)
users_count = len(users_all)
print(f'{users_count} users found...')

95 users found...


In [12]:
#Iterate through each user, determine various properties, and create a list of dictionary
#Refer to properties of User Object: https://developers.arcgis.com/rest/users-groups-and-items/user.htm
f_user = []
attributeMap = ['username','id','fullName','availableCredits','assignedCredits','preferredView', 'email','lastLogin','mfaEnabled','access',
                'orgId','role','privileges','userLicenseTypeId','disabled', 'region','thumbnail','created','modified','groups','provider']

for user in users_all:
    f = {"attributes":{}}
    for field in attributeMap: 
        f['attributes'][field] = user[field]
        
        #Format privileges and add to the dictionary
        if field == 'privileges':
            privilege = user[field]
            privilege_ls = []
            i = 0
            while i < len(user[field]): #Since privileges assigned varies with each user, find the length of the list.
                privilege_str = user[field][i] #Take the i-th item in the list 
                privilege_name = privilege_str.rsplit(':',1)[1] #Format from [portal]:[usertype]:[privilege] to [privilege]
                privilege_ls.append(privilege_name) #Append the formatted string to a list
                i = i + 1
            f['attributes']['user_privileges'] = str(privilege_ls).strip('[]').replace('\'','') #Remove the brackets and quotation mark to insert the string to the dictionary.
            del f['attributes'][field]                                                          #Privilege is a SQL keyword so change to user_privileges. Delete Privileges key from the dict.
        
        #Count the number of groups
        if field == 'groups':
            group_count = len(user[field])
            f['attributes'][field] = group_count
            
    #Count the number of items owned by each user
    item_count = len(user.items(max_items = 1000)) #Max_item is defaulted to 100
    f['attributes']['item'] = item_count
    
    #Determine My Esri access status
    myesri_access = user.esri_access
    f['attributes']['myEsri'] = myesri_access
    
    #Append the record to the list 
    f_user.append(f)

#Display the first 5 items as output
f_user[:1]

[{'attributes': {'username': 'AGiron_aid',
   'id': 'a64c4d9e27704dc2929b83a68947b436',
   'fullName': 'Amanda Giron',
   'availableCredits': 1000.0,
   'assignedCredits': 1000.0,
   'preferredView': None,
   'email': 'AGiron@esri.com',
   'lastLogin': 1607967224000,
   'mfaEnabled': False,
   'access': 'org',
   'orgId': 'LG9Yn2oFqZi5PnO5',
   'role': 'org_publisher',
   'user_privileges': 'edit, bulkPublishFromDataStores, publishDynamicImagery, publishFeatures, publishScenes, publishServerServices, publishTiledImagery, publishTiles, registerDataStores, categorizeItems, createGroup, createItem, joinGroup, joinNonOrgGroup, shareGroupToOrg, shareGroupToPublic, shareToGroup, shareToOrg, shareToPublic, viewOrgGroups, viewOrgItems, viewOrgUsers, geoanalytics, demographics, elevation, featurereport, geocode, stored, temporary, geoenrichment, networkanalysis, closestfacility, locationallocation, optimizedrouting, origindestinationcostmatrix, routing, servicearea, vehiclerouting, spatialanaly

In [13]:
#If this is the first time running the notebook, create the hosted table with the following CSV. 
#Reference the instructions outlined in the blog

#Make sure to set the field types correctly.
#String: username, id, fullName, preferredView, email, mfaEnabled, access, orgId, role, privileges, userLicenseTypeId, disabled, region, thumbnail, provider, myEsri
#Integer: groups, item
#Double: availableCredits, assignedCredits
#Date: lastLogin, created, modified

dest_fl = gis.content.get(table_user).tables[0]
dest_fl

<Table url:"https://services.arcgis.com/LG9Yn2oFqZi5PnO5/arcgis/rest/services/user_details/FeatureServer/0">

In [14]:
#Update the Hosted Table with the item details
dest_fl.manager.truncate()
result = dest_fl.edit_features(adds=f_user)
print('success...')

success...


# Hosted Feature Service Tracking

In [1]:
# @Purpose: Hosted feature service is the most versatile and frequently used type of layer in ArcGIS Online.
# But with the versatility comes cost storing the layers and they are generally the leading source of credit consumption.
# This notebook is meant to be run once a day to keep a running history of feature service usage in the organization 
# in a format that is easy to digest and explore, enabling the administrator to maintain oversight of usage. 
# LIMITATION: The current script does not properly account for FS Storage with attachments. As a result, the calculated credit consumption will be higher than actual. 

In [16]:
#Take the f_item generated from the previous steps and create a new list with type = Feature Service
fl_lst = []
date_today = datetime.datetime.today().strftime('%Y/%m/%d') #Date when the code is run 

for items in f_item:
    if items['attributes']['type'] == 'Feature Service':
        items['attributes']['date_updated'] = date_today
        items['attributes']['credit_cost'] = float(items['attributes']['size_mb']) * .24 #Convert storage to credit cost
        items['attributes']['remaining_credits'] =  gis.admin.credits.credits #Get the remaining credits in the organization
        fl_lst.append(items)

#Add the user's last login information to the item
for fl in fl_lst:
    item_owner = fl['attributes']['owner']
    
    for user in f_user:
        username = user['attributes']['username'] 
        
        if item_owner == username:
            fl['attributes']['last_login'] = user['attributes']['lastLogin'] 

#Summarize the results
fl_count = len(fl_lst)
print(f'{fl_count} feature layers found in the organization...')
fl_lst[:1]

1262 feature layers found in the organization...


[{'attributes': {'id': 'd3bf9513eb6148e99df2436563fe3167',
   'title': 'SIDW_Forum_Sponsors',
   'owner': 'ssawaya_aid',
   'fullname': 'Salim Sawaya',
   'created': '2013-06-02 16:57:55.0',
   'modified': '2013-06-06 06:07:16.0',
   'url': 'http://services.arcgis.com/LG9Yn2oFqZi5PnO5/arcgis/rest/services/SIDW_Forum_Sponsors/FeatureServer',
   'size': 57344,
   'size_mb': '0.057344',
   'access': 'public',
   'type': 'Feature Service',
   'snippet': '',
   'categories': '',
   'numComments': 0,
   'numRatings': 0,
   'numViews': 176,
   'scoreCompleteness': 33,
   'tags': "'SIDW'",
   'tag_status': 'no',
   'date_updated': '2021/04/09',
   'credit_cost': 0.01376256,
   'remaining_credits': 49655.895,
   'last_login': 1611754636000}}]

In [17]:
#If this is the first time running the notebook, create the hosted table with the following CSV. 
#Reference the instructions outlined in the blog

#Make sure to set the field types correctly.
#String: id, title, owner, fullname, url, access, type, snippet, categories, tags, tag_status
#Integer: numComments, numRatings, numViews, scoreCompleteness
#Double: size, size_mb, remaining credits
#Date: created, modified, date_updated, last_login

dest_fl = gis.content.get(table_fs).tables[0]
dest_fl

<Table url:"https://services.arcgis.com/LG9Yn2oFqZi5PnO5/arcgis/rest/services/fl_details/FeatureServer/0">

In [18]:
#Calculate the date in which all records that come before will be deleted
date_result = datetime.date.today() - datetime.timedelta(days=days)
date_retain = date_result.strftime('%Y/%m/%d')
print(f'Any records before {date_retain} will be deleted...')

Any records before 2021/03/10 will be deleted...


In [19]:
#Update the Hosted Table with the item details
dest_fl.delete_features(where=f'date_updated < \'{date_retain}\'') #Delete any records that are older than date_retain
dest_fl.delete_features(where=f'date_updated = \'{date_today}\'') #If the notebook is run more than once a day, delete any previous records from the same day
result = dest_fl.edit_features(adds=fl_lst)
print('success...')

success...


# Licensing

In [21]:
# '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
#    Copyright © 2023 Esri
#
#    All rights reserved under the copyright laws of the United States 
#    and applicable international laws, treaties, and conventions.
#    You may freely redistribute and use this sample code, with or 
#    without modification, provided you include the original copyright 
#    notice and use restrictions.
#
#    Disclaimer: THE SAMPLE CODE IS PROVIDED "AS IS" AND ANY EXPRESS 
#    OR IMPLIED WARRANTIES, INCLUDING THE IMPLIED WARRANTIES OF 
#    MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE 
#    DISCLAIMED. IN NO EVENT SHALL ESRI OR CONTRIBUTORS BE LIABLE FOR
#    ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL 
#    DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS 
#    OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) 
#    SUSTAINED BY YOU OR A THIRD PARTY, HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, 
#    WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT ARISING IN ANY WAY OUT OF THE USE 
#    OF THIS SAMPLE CODE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
#    For additional information, contact:
#    Esri
#    Attn: Contracts and Legal Services Department
#    380 New York Street
#    Redlands, California, 92373-8100
#    USA
#    email: contracts@esri.com
# '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''